In [1]:
from db import fetch
from tqdm import tqdm

In [6]:
query = """
SELECT contests.id, contests.__updatetime, contests.type, contests_plrsbefore.plr, contests.winner, contests.loser, contests_plrsbefore.userid, contests.status

FROM contests 

INNER JOIN contests_plrsbefore 

ON contests_plrsbefore.contests_id = contests.id

WHERE contests_plrsbefore.userid = '7995092012829590'
"""

In [ ]:
df = fetch(query)
df.shape

In [4]:
df.head()

,id,__updatetime,type,plr,winner,loser,userid,status
0,7995092012829590,2020-08-21 19:15:34.342,H,64,4.439926e+14,7.038696e+15,443992607815436,completed
1,5179645355963421,2020-08-21 20:10:45.923,H,59,7.893768e+15,6.613301e+15,6613300821617026,completed
2,5484846488566522,2020-08-21 20:10:45.923,H,64,NaN,NaN,443992607815436,cancelled
3,2230477145682123,2020-08-21 21:13:48.940,H,69,NaN,NaN,1768930121316690,cancelled
4,3214806079020904,2020-08-21 22:19:37.525,H,62,NaN,NaN,4855353181157037,cancelled


In [9]:
lookup = {}

for _, row in tqdm(df.iterrows()):
    if row['status'] != "completed" or row['type'] != 'H':
        pass
    else:
        if row['userid'] not in lookup.keys():
            if row['userid'] == row['winner']:
                lookup[row['userid']] = [(row['__updatetime'], 1)]
            else:
                lookup[row['userid']] = [(row['__updatetime'], 0)]
        else:
            if row['userid'] == row['winner']:
                lookup[row['userid']].append((row['__updatetime'], 1))
            else:
                lookup[row['userid']].append((row['__updatetime'], 0))

333684it [00:45, 7408.74it/s]


In [10]:
# sort games
for k, v in lookup.items():
    lookup[k] = sorted(v, key=lambda x: x[0])

In [11]:
# remove timespace
for k, v in lookup.items():
    lookup[k] = list(map(lambda x: x[1], v))

In [12]:
# insert end 
for k, v in lookup.items():
    lookup[k].append("*")
lookup

{7038695774196162: [0,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  '*'],
 7893767988794126: [1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
 

In [13]:
class Step(object):
    def __init__(self, game_idx, outcome, path):
        self.game_idx = game_idx
        self.count = 0
        self.outcome = outcome
        self.left = None
        self.right = None
        self.end = None
        self.path = path

In [14]:
root = Step("ftd", outcome="start", path="")

data = lookup.values()

for d in data:
    curr = root

    for i, o in enumerate(d):
        if len(d) > 10:
            pass
        
        # scenario 1- win
        elif o == 1:
            if curr.right is None:  
                p = curr.path
                curr.right = Step(game_idx=i, outcome="win", path=p+'W')
            curr = curr.right
            curr.count += 1
        elif o == 0:
            if curr.left is None:
                p = curr.path
                curr.left = Step(i, outcome="loss", path=p+"L")
            curr = curr.left
            curr.count += 1
        else:
            if curr.end is None:
                p = curr.path
                curr.end = Step(i, outcome="end", path=p+'*')
            curr = curr.end
            curr.count += 1

In [15]:
journeys = {}

def f(x):

    if x.outcome == "end":
        print(x.game_idx, x.path, x.count)
        journeys[x.count] = x.path
    else:
        if x.left is not None:
            f(x.left)
        if x.right is not None:
            f(x.right)
        if x.end is not None:
            f(x.end)

In [16]:
f(root)

9 LLLLLLLLL* 11
8 LLLLLLLL* 9
9 LLLLLLLWL* 4
9 LLLLLLLWW* 1
8 LLLLLLLW* 2
7 LLLLLLL* 21
9 LLLLLLWLL* 2
9 LLLLLLWWL* 1
9 LLLLLLWWW* 2
8 LLLLLLWW* 1
7 LLLLLLW* 5
6 LLLLLL* 30
8 LLLLLWLL* 3
9 LLLLLWLWL* 1
7 LLLLLWL* 7
9 LLLLLWWLW* 1
8 LLLLLWWL* 2
8 LLLLLWWW* 1
7 LLLLLWW* 1
6 LLLLLW* 3
5 LLLLL* 49
8 LLLLWLLL* 2
9 LLLLWLLWW* 1
7 LLLLWLL* 5
8 LLLLWLWL* 2
9 LLLLWLWWL* 2
8 LLLLWLWW* 2
7 LLLLWLW* 3
6 LLLLWL* 8
9 LLLLWWLLL* 1
9 LLLLWWLLW* 1
9 LLLLWWLWL* 1
9 LLLLWWWLL* 2
9 LLLLWWWWW* 1
7 LLLLWWW* 1
6 LLLLWW* 1
5 LLLLW* 8
4 LLLL* 105
9 LLLWLLLLL* 1
8 LLLWLLLL* 3
9 LLLWLLLWL* 2
8 LLLWLLLW* 2
7 LLLWLLL* 8
8 LLLWLLWL* 1
6 LLLWLL* 13
9 LLLWLWLLL* 1
8 LLLWLWLL* 3
9 LLLWLWLWW* 1
7 LLLWLWL* 2
9 LLLWLWWLW* 1
9 LLLWLWWWW* 1
6 LLLWLW* 4
5 LLLWL* 14
8 LLLWWLLL* 3
8 LLLWWLLW* 1
7 LLLWWLL* 2
9 LLLWWLWLL* 1
9 LLLWWLWWW* 1
7 LLLWWLW* 3
6 LLLWWL* 3
9 LLLWWWLLL* 1
9 LLLWWWLLW* 1
8 LLLWWWLL* 2
7 LLLWWWL* 2
9 LLLWWWWWL* 1
8 LLLWWWWW* 2
7 LLLWWWW* 1
6 LLLWWW* 2
5 LLLWW* 4
4 LLLW* 13
3 LLL* 194
9 LLWLLLLLL* 2
9 LLWLLL

In [17]:
games_played = sum(journeys.keys())
games_played

2894

In [18]:
for k in sorted(journeys.keys()):
    print(journeys[k], round(k / games_played, 3))

WWWWWWWW* 0.0
WWWWWLWW* 0.001
WWWWWW* 0.001
WWWWWWW* 0.001
WWWWWL* 0.002
WWWLW* 0.002
WWWWW* 0.002
WWWWL* 0.003
WWLLL* 0.003
WWWLL* 0.003
WLWLL* 0.004
WLLWL* 0.004
LLLW* 0.004
WWWW* 0.005
WWWL* 0.006
WWLL* 0.006
LWW* 0.007
WWW* 0.009
LLLLLL* 0.01
WLWL* 0.011
LLWL* 0.012
WLLL* 0.013
LLW* 0.016
LWLL* 0.017
WW* 0.019
WWL* 0.02
LWL* 0.023
LW* 0.027
WLL* 0.031
LLLL* 0.036
WL* 0.053
W* 0.063
LLL* 0.067
LL* 0.174
L* 0.345
